Image classfication using CNN Networks

Authors: Keyon Assadi, Jagadeesh Meesala, Tapan Katipelli, Mihir Palav

# Download dataset from Google Drive

In [ ]:
import os
import gdown #pip3 install gdown
# os.system('!gdown 1_WAm7yPRPK9at-2a2zi1VsiZZq9JLpiz')
# download normal, benign and malignant categorical data
!gdown 1_WAm7yPRPK9at-2a2zi1VsiZZq9JLpiz

Downloading...
From: https://drive.google.com/uc?id=1_WAm7yPRPK9at-2a2zi1VsiZZq9JLpiz
To: /content/The IQ-OTHNCCD lung cancer dataset.zip
100% 157M/157M [00:01<00:00, 122MB/s]


In [ ]:
# multi category data
!unzip "The IQ-OTHNCCD lung cancer dataset.zip"

Archive:  The IQ-OTHNCCD lung cancer dataset.zip
   creating: The IQ-OTHNCCD lung cancer dataset/
  inflating: __MACOSX/._The IQ-OTHNCCD lung cancer dataset  
   creating: The IQ-OTHNCCD lung cancer dataset/Bengin cases/
  inflating: __MACOSX/The IQ-OTHNCCD lung cancer dataset/._Bengin cases  
  inflating: The IQ-OTHNCCD lung cancer dataset/.DS_Store  
  inflating: __MACOSX/The IQ-OTHNCCD lung cancer dataset/._.DS_Store  
   creating: The IQ-OTHNCCD lung cancer dataset/Normal cases/
  inflating: __MACOSX/The IQ-OTHNCCD lung cancer dataset/._Normal cases  
   creating: The IQ-OTHNCCD lung cancer dataset/Malignant cases/
  inflating: __MACOSX/The IQ-OTHNCCD lung cancer dataset/._Malignant cases  
  inflating: The IQ-OTHNCCD lung cancer dataset/IQ-OTH_NCCD lung cancer dataset.txt  
  inflating: The IQ-OTHNCCD lung cancer dataset/Bengin cases/Bengin case (85).jpg  
  inflating: __MACOSX/The IQ-OTHNCCD lung cancer dataset/Bengin cases/._Bengin case (85).jpg  
  inflating: The IQ-OTHNCCD lun

# Required imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.backend import tf as ktf
from keras.callbacks import TensorBoard
from datetime import datetime
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
import tensorboard
tensorboard.__version__

'2.8.0'

In [ ]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, histogram_freq=1)

# Multi classifier model implementation

### Load images using an ImageDataGenerator. With this object we also apply augmentation

### We will create two copies of the training and validation datasets. One will contain Data Augmentation while the others won't

In [ ]:
datagen = ImageDataGenerator(validation_split=0.2)

datagen_ia = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True,
    fill_mode = 'nearest',
    validation_split=0.2)

train_generator = datagen.flow_from_directory('The IQ-OTHNCCD lung cancer dataset/',
                                              target_size = (300,300),
                                              class_mode = 'categorical',
                                              subset='training')

test_generator = datagen.flow_from_directory('The IQ-OTHNCCD lung cancer dataset/',
                                              target_size = (300,300),
                                              class_mode = 'categorical',
                                              subset='validation')

train_generator_ia = datagen_ia.flow_from_directory('The IQ-OTHNCCD lung cancer dataset/',
                                              target_size = (300,300),
                                              class_mode = 'categorical',
                                              subset='training')

test_generator_ia = datagen_ia.flow_from_directory('The IQ-OTHNCCD lung cancer dataset/',
                                              target_size = (300,300),
                                              class_mode = 'categorical',
                                              subset='validation')

Found 878 images belonging to 3 classes.
Found 219 images belonging to 3 classes.
Found 878 images belonging to 3 classes.
Found 219 images belonging to 3 classes.


### Base Model

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten()) # Flattens the input to 1D
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(1024, activation = 'relu'))
model.add(tf.keras.layers.Dense(3, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy', tf.keras.metrics.Recall()])

history1 = model.fit(
      train_generator,
      epochs=20,
      verbose=1,
      validation_data = test_generator)

Epoch 1/20
28/28 [==============================] - 33s 1s/step - loss: 14295.0576 - accuracy: 0.4727 - recall: 0.4727 - val_loss: 3454.9565 - val_accuracy: 0.5251 - val_recall: 0.5251
Epoch 2/20
28/28 [==============================] - 33s 1s/step - loss: 525.9400 - accuracy: 0.7392 - recall: 0.7392 - val_loss: 1153.1317 - val_accuracy: 0.5388 - val_recall: 0.5388
Epoch 3/20
28/28 [==============================] - 32s 1s/step - loss: 303.8318 - accuracy: 0.7346 - recall: 0.7346 - val_loss: 793.1229 - val_accuracy: 0.4384 - val_recall: 0.4384
Epoch 4/20
28/28 [==============================] - 32s 1s/step - loss: 281.3525 - accuracy: 0.7677 - recall: 0.7677 - val_loss: 1290.4154 - val_accuracy: 0.4795 - val_recall: 0.4795
Epoch 5/20
28/28 [==============================] - 30s 1s/step - loss: 105.1599 - accuracy: 0.8440 - recall: 0.8440 - val_loss: 668.9291 - val_accuracy: 0.4521 - val_recall: 0.4521
Epoch 6/20
28/28 [==============================] - 32s 1s/step - loss: 96.9712 - acc

### CNN Model without Data Augmentation

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(512, activation = 'relu'),
     tf.keras.layers.Dense(1024, activation = 'relu'),
     tf.keras.layers.Dense(3, activation = 'softmax')
    ]
)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy',tf.keras.metrics.Recall()])
history2 = model.fit(
      train_generator,
      epochs=20,
      verbose=1,
      validation_data = test_generator)

Epoch 1/20
28/28 [==============================] - 93s 3s/step - loss: 16.1601 - accuracy: 0.4487 - recall_1: 0.3838 - val_loss: 1.0394 - val_accuracy: 0.6256 - val_recall_1: 0.5434
Epoch 2/20
28/28 [==============================] - 93s 3s/step - loss: 0.8398 - accuracy: 0.6241 - recall_1: 0.5159 - val_loss: 0.9154 - val_accuracy: 0.5662 - val_recall_1: 0.5342
Epoch 3/20
28/28 [==============================] - 93s 3s/step - loss: 0.6514 - accuracy: 0.7267 - recall_1: 0.6606 - val_loss: 1.0035 - val_accuracy: 0.5936 - val_recall_1: 0.5571
Epoch 4/20
28/28 [==============================] - 92s 3s/step - loss: 0.3760 - accuracy: 0.8679 - recall_1: 0.8371 - val_loss: 1.0479 - val_accuracy: 0.6621 - val_recall_1: 0.6393
Epoch 5/20
28/28 [==============================] - 92s 3s/step - loss: 0.2743 - accuracy: 0.8918 - recall_1: 0.8759 - val_loss: 1.3421 - val_accuracy: 0.6164 - val_recall_1: 0.6027
Epoch 6/20
28/28 [==============================] - 91s 3s/step - loss: 0.1276 - accuracy

### CNN Model with Data Augmentation

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(512, activation = 'relu'),
     tf.keras.layers.Dense(1024, activation = 'relu'),
     tf.keras.layers.Dense(3, activation = 'softmax')
    ]
)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy',tf.keras.metrics.Recall()])
history3 = model.fit(
      train_generator_ia,
      epochs=20,
      verbose=1,
      validation_data = test_generator_ia)
      # pass tracing
  #    callbacks=[tensorboard_callback])

Epoch 1/20
28/28 [==============================] - 117s 4s/step - loss: 0.9894 - accuracy: 0.4863 - recall_2: 0.2631 - val_loss: 0.9656 - val_accuracy: 0.5114 - val_recall_2: 0.5114
Epoch 2/20
28/28 [==============================] - 110s 4s/step - loss: 0.9148 - accuracy: 0.5524 - recall_2: 0.3576 - val_loss: 0.9672 - val_accuracy: 0.5479 - val_recall_2: 0.5114
Epoch 3/20
28/28 [==============================] - 109s 4s/step - loss: 0.7670 - accuracy: 0.6856 - recall_2: 0.6082 - val_loss: 1.0424 - val_accuracy: 0.5753 - val_recall_2: 0.5160
Epoch 4/20
28/28 [==============================] - 111s 4s/step - loss: 0.7132 - accuracy: 0.7005 - recall_2: 0.6287 - val_loss: 0.9290 - val_accuracy: 0.6256 - val_recall_2: 0.5068
Epoch 5/20
28/28 [==============================] - 109s 4s/step - loss: 0.6886 - accuracy: 0.7335 - recall_2: 0.6640 - val_loss: 1.0185 - val_accuracy: 0.6027 - val_recall_2: 0.5753
Epoch 6/20
28/28 [==============================] - 116s 4s/step - loss: 0.6045 - acc

In [ ]:
%tensorboard --logdir logs/fit

<IPython.core.display.Javascript object>

### CNN Model with Data Augmentation and Normalization

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(512, activation = 'relu'),
     tf.keras.layers.Dense(1024, activation = 'relu'),
     tf.keras.layers.Dropout(0.3),
     tf.keras.layers.Dense(3, activation = 'softmax')
    ]
)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy',tf.keras.metrics.Recall()])
history4 = model.fit(
      train_generator_ia,
      epochs=20,
      verbose=1,
      validation_data = test_generator_ia)

Epoch 1/20
28/28 [==============================] - 121s 4s/step - loss: 1.3721 - accuracy: 0.6298 - recall_3: 0.6059 - val_loss: 1.8872 - val_accuracy: 0.5114 - val_recall_3: 0.5114
Epoch 2/20
28/28 [==============================] - 116s 4s/step - loss: 0.6978 - accuracy: 0.7301 - recall_3: 0.7130 - val_loss: 8.9387 - val_accuracy: 0.5114 - val_recall_3: 0.5114
Epoch 3/20
28/28 [==============================] - 120s 4s/step - loss: 0.5033 - accuracy: 0.8087 - recall_3: 0.7847 - val_loss: 6.2844 - val_accuracy: 0.5114 - val_recall_3: 0.5114
Epoch 4/20
28/28 [==============================] - 117s 4s/step - loss: 0.3813 - accuracy: 0.8440 - recall_3: 0.8303 - val_loss: 18.4802 - val_accuracy: 0.5114 - val_recall_3: 0.5114
Epoch 5/20
28/28 [==============================] - 115s 4s/step - loss: 0.4350 - accuracy: 0.8394 - recall_3: 0.8235 - val_loss: 43.5431 - val_accuracy: 0.5114 - val_recall_3: 0.5114
Epoch 6/20
28/28 [==============================] - 116s 4s/step - loss: 0.4537 - a

### LSTM Network

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Reshape((7,7*256)),
     tf.keras.layers.LSTM(128),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(512, activation = 'relu'),
     tf.keras.layers.Dense(1024, activation = 'relu'),
     tf.keras.layers.Dropout(0.3),
     tf.keras.layers.Dense(3, activation = 'softmax')
    ]
)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy',tf.keras.metrics.Recall()])
history5 = model.fit(
      train_generator_ia,
      epochs=20,
      verbose=1,
      validation_data = test_generator_ia)

Epoch 1/20
28/28 [==============================] - 122s 4s/step - loss: 0.6867 - accuracy: 0.7118 - recall_4: 0.6526 - val_loss: 0.9576 - val_accuracy: 0.4155 - val_recall_4: 0.2009
Epoch 2/20
28/28 [==============================] - 118s 4s/step - loss: 0.4843 - accuracy: 0.7995 - recall_4: 0.7563 - val_loss: 0.9241 - val_accuracy: 0.4110 - val_recall_4: 0.3927
Epoch 3/20
28/28 [==============================] - 124s 4s/step - loss: 0.4018 - accuracy: 0.8292 - recall_4: 0.7984 - val_loss: 0.8496 - val_accuracy: 0.6073 - val_recall_4: 0.5342
Epoch 4/20
28/28 [==============================] - 119s 4s/step - loss: 0.3990 - accuracy: 0.8292 - recall_4: 0.8052 - val_loss: 3.0140 - val_accuracy: 0.3790 - val_recall_4: 0.3790
Epoch 5/20
28/28 [==============================] - 117s 4s/step - loss: 0.4449 - accuracy: 0.8269 - recall_4: 0.7973 - val_loss: 2.4698 - val_accuracy: 0.3790 - val_recall_4: 0.3790
Epoch 6/20
28/28 [==============================] - 115s 4s/step - loss: 0.3730 - acc

### GoogLeNet
GooLeNet consist of 22 layers, 9 inception blocks and 3 softmax outputs

Reference: https://medium.com/mlearning-ai/implementation-of-googlenet-on-keras-d9873aeed83c

In [ ]:
def inception_block(input, path1_filters, path2_filters, path3_filters, path4_filters):
  # First path:
  path1 = tf.keras.layers.Conv2D(filters = path1_filters, kernel_size = (1,1), padding = 'same', activation = 'relu')(input)

  # Second path
  path2 = tf.keras.layers.Conv2D(filters = path2_filters[0], kernel_size = (1,1), padding = 'same', activation = 'relu')(input)
  path2 = tf.keras.layers.Conv2D(filters = path2_filters[1], kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # Third path
  path3 = tf.keras.layers.Conv2D(filters = path3_filters[0], kernel_size = (1,1), padding = 'same', activation = 'relu')(input)
  path3 = tf.keras.layers.Conv2D(filters = path3_filters[1], kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = tf.keras.layers.MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input)
  path4 = tf.keras.layers.Conv2D(filters = path4_filters, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = tf.keras.layers.concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer

In [ ]:
def GoogLeNet():
  # Define the input layer with an input size equal to the size of the images
  input = tf.keras.layers.Input(shape = (300, 300, 3))

  # First Conv2D layer: filters = 64, kernel_size = (7,7), strides = 2
  X = tf.keras.layers.Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # convolutional layer: filters = 64, strides = 1
  X = tf.keras.layers.Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

  # convolutional layer: filters = 192, kernel_size = (3,3)
  X = tf.keras.layers.Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = tf.keras.layers.MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 1st Inception block
  X = inception_block(X, path1_filters = 64, path2_filters = (96, 128), path3_filters = (16, 32), path4_filters = 32)

  # 2nd Inception block
  X = inception_block(X, path1_filters = 128, path2_filters = (128, 192), path3_filters = (32, 96), path4_filters = 64)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = tf.keras.layers.MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 3rd Inception block
  X = inception_block(X, path1_filters = 192, path2_filters = (96, 208), path3_filters = (16, 48), path4_filters = 64)

  # Extra network 1:
  X1 = tf.keras.layers.AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
  X1 = tf.keras.layers.Flatten()(X1)
  X1 = tf.keras.layers.Dense(1024, activation = 'relu')(X1)
  X1 = tf.keras.layers.Dropout(0.7)(X1)
  X1 = tf.keras.layers.Dense(3, activation = 'softmax', name = 'Extra_Network_1')(X1)

  
  # 4th Inception block
  X = inception_block(X, path1_filters = 160, path2_filters = (112, 224), path3_filters= (24, 64), path4_filters = 64)

  # 5th Inception block
  X = inception_block(X, path1_filters = 128, path2_filters = (128, 256), path3_filters = (24, 64), path4_filters = 64)

  # 6th Inception block
  X = inception_block(X, path1_filters = 112, path2_filters = (144, 288), path3_filters = (32, 64), path4_filters = 64)

  # Extra network 2:
  X2 = tf.keras.layers.AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
  X2 = tf.keras.layers.Flatten()(X2)
  X2 = tf.keras.layers.Dense(1024, activation = 'relu')(X2)
  X2 = tf.keras.layers.Dropout(0.7)(X2)
  X2 = tf.keras.layers.Dense(3, activation = 'softmax', name = 'Extra_Network_2')(X2)
  
  
  # 7th Inception block
  X = inception_block(X, path1_filters = 256, path2_filters = (160, 320), path3_filters = (32, 128), path4_filters = 128)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # 8th Inception block
  X = inception_block(X, path1_filters = 256, path2_filters = (160, 320), path3_filters = (32, 128), path4_filters = 128)

  # 9th Inception block
  X = inception_block(X, path1_filters = 384, path2_filters = (192, 384), path3_filters = (48, 128), path4_filters = 128)

  # Global Average pooling layer 
  X = tf.keras.layers.GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = tf.keras.layers.Dropout(0.4)(X)

  # output layer 
  X = tf.keras.layers.Dense(3, activation = 'softmax', name = 'Output')(X)
  
  # model
  model = tf.keras.Model(input, [X, X1, X2], name = 'GoogLeNet')

  return model

In [ ]:
model = GoogLeNet()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy',tf.keras.metrics.Recall()])
history = model.fit(train_generator_ia, epochs = 20, verbose=1, validation_data = test_generator_ia)

### Plot accuracies and losses

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize=(15,15))
axes[0,0].plot(history1.history['accuracy'], label = 'Base model')
axes[0,0].plot(history2.history['accuracy'], label = 'CNN model without DA')
axes[0,0].plot(history3.history['accuracy'], label = 'CNN model with DA')
axes[0,0].plot(history4.history['accuracy'], label = 'CNN model with DA and Norm.')
axes[0,0].plot(history5.history['accuracy'], label = 'LSTM model')
axes[0,0].plot(history.history['Output_accuracy'], label = 'GoogLeNet')
axes[0,0].set_xlabel('Epoch')
axes[0,0].set_ylabel('Accuracy')

axes[1,0].plot(history1.history['val_accuracy'], label = 'Base model')
axes[1,0].plot(history2.history['val_accuracy'], label = 'CNN model without DA')
axes[1,0].plot(history3.history['val_accuracy'], label = 'CNN model with DA')
axes[1,0].plot(history4.history['val_accuracy'], label = 'CNN model with DA and Norm.')
axes[1,0].plot(history5.history['val_accuracy'], label = 'LSTM model')
axes[0,0].plot(history.history['val_Output_accuracy'], label = 'GoogLeNet')
axes[1,0].set_xlabel('Epoch')
axes[1,0].set_ylabel('Val. Accuracy')

axes[0,1].plot(history1.history['loss'], label = 'Base model')
axes[0,1].plot(history2.history['loss'], label = 'CNN model without DA')
axes[0,1].plot(history3.history['loss'], label = 'CNN model with DA')
axes[0,1].plot(history4.history['loss'], label = 'CNN model with DA and Norm.')
axes[0,1].plot(history5.history['loss'], label = 'LSTM model')
axes[0,0].plot(history.history['Output_loss'], label = 'GoogLeNet')
axes[0,1].set_xlabel('Epoch')
axes[0,1].set_ylabel('Loss')

axes[1,1].plot(history1.history['val_loss'], label = 'Base model')
axes[1,1].plot(history2.history['val_loss'], label = 'CNN model without DA')
axes[1,1].plot(history3.history['val_loss'], label = 'CNN model with DA')
axes[1,1].plot(history4.history['val_loss'], label = 'CNN model with DA and Norm.')
axes[1,1].plot(history5.history['val_loss'], label = 'LSTM model')
axes[0,0].plot(history.history['val_Output_loss'], label = 'GoogLeNet')
axes[1,1].set_xlabel('Epoch')
axes[1,1].set_ylabel('Val. Loss')

plt.legend()
plt.show()

We can see that the CNN model with Data Augmentation is the one with the highest accuracies (training and validation accuracies) and the smallest loss values.

# Binary Classifier implementation -

In [ ]:
# binary classifier malignant/non-malignant data
!gdown 1lhWypCSTFOECH_9R3WseFeBBse9pXH5A
# https://drive.google.com/file/d/1lhWypCSTFOECH_9R3WseFeBBse9pXH5A/view?usp=sharing
# https://drive.google.com/file/d/1lhWypCSTFOECH_9R3WseFeBBse9pXH5A/view?usp=sharing

In [ ]:
# binary category data
!unzip "binary_classifier_data.zip"

### Load images using an ImageDataGenerator. With this object we also apply augmentation

### We will create two copies of the training and validation datasets. One will contain Data Augmentation while the others won't

In [ ]:
datagen = ImageDataGenerator(validation_split=0.2)

datagen_ia = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True,
    fill_mode = 'nearest',
    validation_split=0.2)

train_generator = datagen.flow_from_directory('binary_classifier_data/',
                                              target_size = (300,300),
                                              class_mode = 'categorical',
                                              subset='training')

test_generator = datagen.flow_from_directory('binary_classifier_data/',
                                              target_size = (300,300),
                                              class_mode = 'categorical',
                                              subset='validation')

train_generator_ia = datagen_ia.flow_from_directory('binary_classifier_data/',
                                              target_size = (300,300),
                                              class_mode = 'categorical',
                                              subset='training')

test_generator_ia = datagen_ia.flow_from_directory('binary_classifier_data/',
                                              target_size = (300,300),
                                              class_mode = 'categorical',
                                              subset='validation')

### Base Model

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten()) # Flattens the input to 1D
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(1024, activation = 'relu'))
model.add(tf.keras.layers.Dense(2, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

history1 = model.fit(
      train_generator,
      epochs=20,
      verbose=1,
      validation_data = test_generator)

### CNN Model without Data Augmentation

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(512, activation = 'relu'),
     tf.keras.layers.Dense(1024, activation = 'relu'),
     tf.keras.layers.Dense(2, activation = 'softmax')
    ]
)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history2 = model.fit(
      train_generator,
      epochs=20,
      verbose=1,
      validation_data = test_generator)

### CNN Model with Data Augmentation

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(512, activation = 'relu'),
     tf.keras.layers.Dense(1024, activation = 'relu'),
     tf.keras.layers.Dense(2, activation = 'softmax')
    ]
)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history3 = model.fit(
      train_generator_ia,
      epochs=20,
      verbose=1,
      validation_data = test_generator_ia)f

### CNN Model with Data Augmentation and Normalization

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(512, activation = 'relu'),
     tf.keras.layers.Dense(1024, activation = 'relu'),
     tf.keras.layers.Dropout(0.3),
     tf.keras.layers.Dense(2, activation = 'softmax')
    ]
)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history4 = model.fit(
      train_generator_ia,
      epochs=20,
      verbose=1,
      validation_data = test_generator_ia)

### LSTM Network

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Reshape((7,7*256)),
     tf.keras.layers.LSTM(128),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(512, activation = 'relu'),
     tf.keras.layers.Dense(1024, activation = 'relu'),
     tf.keras.layers.Dropout(0.3),
     tf.keras.layers.Dense(2, activation = 'softmax')
    ]
)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history5 = model.fit(
      train_generator_ia,
      epochs=20,
      verbose=1,
      validation_data = test_generator_ia)

### GoogLeNet
GooLeNet consist of 22 layers, 9 inception blocks and 3 softmax outputs

Reference: https://medium.com/mlearning-ai/implementation-of-googlenet-on-keras-d9873aeed83c

In [ ]:
def inception_block(input, path1_filters, path2_filters, path3_filters, path4_filters):
  # First path:
  path1 = tf.keras.layers.Conv2D(filters = path1_filters, kernel_size = (1,1), padding = 'same', activation = 'relu')(input)

  # Second path
  path2 = tf.keras.layers.Conv2D(filters = path2_filters[0], kernel_size = (1,1), padding = 'same', activation = 'relu')(input)
  path2 = tf.keras.layers.Conv2D(filters = path2_filters[1], kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # Third path
  path3 = tf.keras.layers.Conv2D(filters = path3_filters[0], kernel_size = (1,1), padding = 'same', activation = 'relu')(input)
  path3 = tf.keras.layers.Conv2D(filters = path3_filters[1], kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = tf.keras.layers.MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input)
  path4 = tf.keras.layers.Conv2D(filters = path4_filters, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = tf.keras.layers.concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer

In [ ]:
def GoogLeNet():
  # Define the input layer with an input size equal to the size of the images
  input = tf.keras.layers.Input(shape = (300, 300, 3))

  # First Conv2D layer: filters = 64, kernel_size = (7,7), strides = 2
  X = tf.keras.layers.Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # convolutional layer: filters = 64, strides = 1
  X = tf.keras.layers.Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

  # convolutional layer: filters = 192, kernel_size = (3,3)
  X = tf.keras.layers.Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = tf.keras.layers.MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 1st Inception block
  X = inception_block(X, path1_filters = 64, path2_filters = (96, 128), path3_filters = (16, 32), path4_filters = 32)

  # 2nd Inception block
  X = inception_block(X, path1_filters = 128, path2_filters = (128, 192), path3_filters = (32, 96), path4_filters = 64)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = tf.keras.layers.MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 3rd Inception block
  X = inception_block(X, path1_filters = 192, path2_filters = (96, 208), path3_filters = (16, 48), path4_filters = 64)

  # Extra network 1:
  X1 = tf.keras.layers.AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
  X1 = tf.keras.layers.Flatten()(X1)
  X1 = tf.keras.layers.Dense(1024, activation = 'relu')(X1)
  X1 = tf.keras.layers.Dropout(0.7)(X1)
  X1 = tf.keras.layers.Dense(2, activation = 'softmax', name = 'Extra_Network_1')(X1)

  
  # 4th Inception block
  X = inception_block(X, path1_filters = 160, path2_filters = (112, 224), path3_filters= (24, 64), path4_filters = 64)

  # 5th Inception block
  X = inception_block(X, path1_filters = 128, path2_filters = (128, 256), path3_filters = (24, 64), path4_filters = 64)

  # 6th Inception block
  X = inception_block(X, path1_filters = 112, path2_filters = (144, 288), path3_filters = (32, 64), path4_filters = 64)

  # Extra network 2:
  X2 = tf.keras.layers.AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
  X2 = tf.keras.layers.Flatten()(X2)
  X2 = tf.keras.layers.Dense(1024, activation = 'relu')(X2)
  X2 = tf.keras.layers.Dropout(0.7)(X2)
  X2 = tf.keras.layers.Dense(2, activation = 'softmax', name = 'Extra_Network_2')(X2)
  
  
  # 7th Inception block
  X = inception_block(X, path1_filters = 256, path2_filters = (160, 320), path3_filters = (32, 128), path4_filters = 128)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # 8th Inception block
  X = inception_block(X, path1_filters = 256, path2_filters = (160, 320), path3_filters = (32, 128), path4_filters = 128)

  # 9th Inception block
  X = inception_block(X, path1_filters = 384, path2_filters = (192, 384), path3_filters = (48, 128), path4_filters = 128)

  # Global Average pooling layer 
  X = tf.keras.layers.GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = tf.keras.layers.Dropout(0.4)(X)

  # output layer 
  X = tf.keras.layers.Dense(2, activation = 'softmax', name = 'Output')(X)
  
  # model
  model = tf.keras.Model(input, [X, X1, X2], name = 'GoogLeNet')

  return model

In [ ]:
model = GoogLeNet()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history = model.fit(train_generator_ia, epochs = 20, verbose=1, validation_data = test_generator_ia)

### Plot accuracies and losses

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize=(15,15))
axes[0,0].plot(history1.history['accuracy'], label = 'Base model')
axes[0,0].plot(history2.history['accuracy'], label = 'CNN model without DA')
axes[0,0].plot(history3.history['accuracy'], label = 'CNN model with DA')
axes[0,0].plot(history4.history['accuracy'], label = 'CNN model with DA and Norm.')
axes[0,0].plot(history5.history['accuracy'], label = 'LSTM model')
axes[0,0].plot(history.history['Output_accuracy'], label = 'GoogLeNet')
axes[0,0].set_xlabel('Epoch')
axes[0,0].set_ylabel('Accuracy')

axes[1,0].plot(history1.history['val_accuracy'], label = 'Base model')
axes[1,0].plot(history2.history['val_accuracy'], label = 'CNN model without DA')
axes[1,0].plot(history3.history['val_accuracy'], label = 'CNN model with DA')
axes[1,0].plot(history4.history['val_accuracy'], label = 'CNN model with DA and Norm.')
axes[1,0].plot(history5.history['val_accuracy'], label = 'LSTM model')
axes[0,0].plot(history.history['val_Output_accuracy'], label = 'GoogLeNet')
axes[1,0].set_xlabel('Epoch')
axes[1,0].set_ylabel('Val. Accuracy')

axes[0,1].plot(history1.history['loss'], label = 'Base model')
axes[0,1].plot(history2.history['loss'], label = 'CNN model without DA')
axes[0,1].plot(history3.history['loss'], label = 'CNN model with DA')
axes[0,1].plot(history4.history['loss'], label = 'CNN model with DA and Norm.')
axes[0,1].plot(history5.history['loss'], label = 'LSTM model')
axes[0,0].plot(history.history['Output_loss'], label = 'GoogLeNet')
axes[0,1].set_xlabel('Epoch')
axes[0,1].set_ylabel('Loss')

axes[1,1].plot(history1.history['val_loss'], label = 'Base model')
axes[1,1].plot(history2.history['val_loss'], label = 'CNN model without DA')
axes[1,1].plot(history3.history['val_loss'], label = 'CNN model with DA')
axes[1,1].plot(history4.history['val_loss'], label = 'CNN model with DA and Norm.')
axes[1,1].plot(history5.history['val_loss'], label = 'LSTM model')
axes[0,0].plot(history.history['val_Output_loss'], label = 'GoogLeNet')
axes[1,1].set_xlabel('Epoch')
axes[1,1].set_ylabel('Val. Loss')

plt.legend()
plt.show()

We can see that the CNN model with Data Augmentation is the one with the highest accuracies (training and validation accuracies) and the smallest loss values.

# Encoder and Decoder
Test with a Denoise encoder

### First, extract the images to split into X and y. We also convert the images to gray scale (we remove convert them from RGB to Grayscale because the images are black and white so no need to use RGB)


**Note: The for-loop has a counter to only extract the first 50 images. Due to Google-Colab is crashing to load all images due to insufficient RAM.**

In [ ]:
train_generator_ia.reset()
X_train, y_train = next(train_generator_ia)
X_train = X_train.mean(axis=3)
X_train = np.expand_dims(X_train, axis = -1)
for i in range(1,50):
  X, y = next(train_generator_ia)
  X = X.mean(axis=3)
  X = np.expand_dims(X, axis = -1)
  X_train = np.append(X_train, X, axis = 0)

test_generator_ia.reset()
X_test, y_test = next(train_generator_ia)
X_test = X_test.mean(axis=3)
X_test = np.expand_dims(X_test, axis = -1)
for i in range(1,50):
  X, y = next(test_generator_ia)
  X = X.mean(axis=3)
  X = np.expand_dims(X, axis = -1)
  X_test = np.append(X_test, X, axis = 0)

In [ ]:
class Denoise(tf.keras.Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
      tf.keras.layers.Input(shape=(300, 300, 1)),
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      tf.keras.layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      tf.keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      tf.keras.layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


In [ ]:
autoencoder = Denoise()
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(X_train, X_train,
                epochs=20,
                shuffle=True,
                validation_data = (X_test, X_test))

In [ ]:
decoded_imgs = autoencoder.predict(X_test)

n = 4
plt.figure(figsize=(20, 4))
for i in range(1, n + 1):
    # Display original
    ax = plt.subplot(2, n, i)
    plt.imshow(tf.squeeze(X_test[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(tf.squeeze(decoded_imgs[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Plot accuracies and losses

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize=(15,15))
axes[0,0].plot(history1.history['accuracy'], label = 'Base model')
axes[0,0].plot(history2.history['accuracy'], label = 'CNN model without DA')
axes[0,0].plot(history3.history['accuracy'], label = 'CNN model with DA')
axes[0,0].plot(history4.history['accuracy'], label = 'CNN model with DA and Norm.')
axes[0,0].plot(history5.history['accuracy'], label = 'LSTM model')
axes[0,0].plot(history.history['Output_accuracy'], label = 'GoogLeNet')
axes[0,0].set_xlabel('Epoch')
axes[0,0].set_ylabel('Accuracy')

axes[1,0].plot(history1.history['val_accuracy'], label = 'Base model')
axes[1,0].plot(history2.history['val_accuracy'], label = 'CNN model without DA')
axes[1,0].plot(history3.history['val_accuracy'], label = 'CNN model with DA')
axes[1,0].plot(history4.history['val_accuracy'], label = 'CNN model with DA and Norm.')
axes[1,0].plot(history5.history['val_accuracy'], label = 'LSTM model')
axes[0,0].plot(history.history['val_Output_accuracy'], label = 'GoogLeNet')
axes[1,0].set_xlabel('Epoch')
axes[1,0].set_ylabel('Val. Accuracy')

axes[0,1].plot(history1.history['loss'], label = 'Base model')
axes[0,1].plot(history2.history['loss'], label = 'CNN model without DA')
axes[0,1].plot(history3.history['loss'], label = 'CNN model with DA')
axes[0,1].plot(history4.history['loss'], label = 'CNN model with DA and Norm.')
axes[0,1].plot(history5.history['loss'], label = 'LSTM model')
axes[0,0].plot(history.history['Output_loss'], label = 'GoogLeNet')
axes[0,1].set_xlabel('Epoch')
axes[0,1].set_ylabel('Loss')

axes[1,1].plot(history1.history['val_loss'], label = 'Base model')
axes[1,1].plot(history2.history['val_loss'], label = 'CNN model without DA')
axes[1,1].plot(history3.history['val_loss'], label = 'CNN model with DA')
axes[1,1].plot(history4.history['val_loss'], label = 'CNN model with DA and Norm.')
axes[1,1].plot(history5.history['val_loss'], label = 'LSTM model')
axes[0,0].plot(history.history['val_Output_loss'], label = 'GoogLeNet')
axes[1,1].set_xlabel('Epoch')
axes[1,1].set_ylabel('Val. Loss')

plt.legend()
plt.show()

We can see that the CNN model with Data Augmentation is the one with the highest accuracies (training and validation accuracies) and the smallest loss values.